# Multi Agent orchestrator using Azure AI Agentic Service

In this example we will summarize a research paper into LinkedIn post using, Azure Agentic Service, Bing & LLMs

In [ ]:
## Project PIP requirements
# %pip install -q azure-ai-projects azure-identity azure-ai-ml azure-search-documents tika "autogen-agentchat" "autogen-ext[openai]"
# %pip install -q -U "autogen-agentchat" "autogen-ext[openai]"

Note: you may need to restart the kernel to use updated packages.


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
opentelemetry-exporter-otlp-proto-grpc 1.26.0 requires opentelemetry-sdk~=1.26.0, but you have opentelemetry-sdk 1.31.0 which is incompatible.
opentelemetry-proto 1.26.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.29.3 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [58]:

# Import required libraries
import os
from dotenv import load_dotenv
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import CodeInterpreterTool
from azure.identity import DefaultAzureCredential
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import AzureOpenAIChatCompletionClient
from autogen_core.models import UserMessage

# Load environment variables from .env file
load_dotenv()

# Research paper path
research_paper_path =  "https://arxiv.org/pdf/2503.05142"

In [96]:
project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

In [97]:
az_model_client = AzureOpenAIChatCompletionClient(
    azure_deployment="gpt-4o-mini",
    api_version="2024-05-01-preview",
    model = "gpt-4o-mini",
    azure_endpoint=os.environ["AOAI_ENDPOINT"], # Azure OpenAI endpoint.
    api_key=os.environ["AOAI_KEY"], # For key-based authentication.
)

# Test the Azure OpenAI model client
result = await az_model_client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

finish_reason='stop' content='The capital of France is Paris.' usage=RequestUsage(prompt_tokens=15, completion_tokens=7) cached=False logprobs=None thought=None


## Create Agents

**Pre-requisites**
1. Create Azure AI Search and configure as connector to the agent. [Link](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/azure-ai-search?tabs=pythonsdk%2Cpython&pivots=code-examples#setup-create-an-agent-that-can-use-an-existing-azure-ai-search-index)

### Download and ingest the file to vector store

In [98]:
import requests
from tika import parser 
# create azure search index
from azure.search.documents.indexes import SearchIndexClient
from azure.core.credentials import AzureKeyCredential
from azure.search.documents.indexes.models import (
    SearchField,
    SearchFieldDataType,
    VectorSearch,
    HnswAlgorithmConfiguration,
    VectorSearchProfile,
    SearchIndex
)
from azure.search.documents import SearchClient

os.makedirs("./data", exist_ok=True)

file_path = "./data/research_paper.pdf"

# Download research paper
if not os.path.exists(file_path):
    print("Downloading research paper...")
    response = requests.get(research_paper_path)
    with open(file_path, "wb") as f:
       f.write(response.content)
else:
    print("Research paper already downloaded.")
    
index_name = "research-paper-index"
index_client = SearchIndexClient(os.environ["SEARCH_ENDPOINT"],AzureKeyCredential(os.environ["SEARCH_KEY"]))
fields = [
    SearchField(name="id", type=SearchFieldDataType.String, key=True),
    SearchField(name="content", type=SearchFieldDataType.String, sortable=False, filterable=False, facetable=False),
]
# Configure the vector search configuration  
vector_search = VectorSearch(  
    algorithms=[  
        HnswAlgorithmConfiguration(name="myHnsw"),
    ],  
    profiles=[  
        VectorSearchProfile(  
            name="myHnswProfile",  
            algorithm_configuration_name="myHnsw",  
        )
    ]
)  
  
# Create the search index
index = SearchIndex(name=index_name, fields=fields, vector_search=vector_search)  
result = index_client.create_or_update_index(index)  
print(f"{result.name} created")
raw = parser.from_file(file_path)
content = raw['content']

search_client = SearchClient(os.environ["SEARCH_ENDPOINT"], index_name, AzureKeyCredential(os.environ["SEARCH_KEY"]))
# Upload the research paper content to the search index
documents = [
    {
        "id": "1",
        "content": content
    }
]
result = search_client.upload_documents(documents=documents)

Research paper already downloaded.
research-paper-index created


### Create Azure AI Search Agent for Custom Knowledge base

In [106]:
from azure.ai.projects.models import AzureAISearchTool

# Upload research paper to Azure AI Search
conn_list = project_client.connections._list_connections()["value"]
conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    metadata = conn["properties"].get("metadata", {})
    if metadata.get("type", "").upper() == "AZURE_AI_SEARCH":
        conn_id = conn["id"]
        break

print(f"Connection ID: {conn_id}")

async def ai_search_tool(query: str):

    ai_search = AzureAISearchTool(index_connection_id=conn_id, index_name=index_name)
    with project_client:
        print(query)
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="research-doc-assistant",
            instructions="""
            You are a research doc search assistant.
            You can search for information in the research paper provided.
            You can also answer questions about the research paper.
            You can also summarize the research paper.
            """,
            tools=ai_search.definitions,
            headers={"x-ms-enable-preview": "true"},
            tool_resources = ai_search.resources,
        )
        
        thread = project_client.agents.create_thread()
        
        print(f"Created thread, ID: {thread.id}")
        
        # Create message to thread
        message = project_client.agents.create_message(
             thread_id=thread.id,
             role="user",
             content=query,
        )
        
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, agent_id=agent.id)
        print(f"Run finished with status: {run.status}")
        if run.status == "failed":
            print(f"Run failed: {run.last_error}")
        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")
        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]
    

Connection ID: /subscriptions/6a01260f-39d6-415f-a6c9-cf4fd479cbec/resourceGroups/rg-vism-7896_ai/providers/Microsoft.MachineLearningServices/workspaces/agentic-service/connections/vectorsearchagentic


In [108]:
ai_search_agent = AssistantAgent(
    name="ai_search_agent",
    model_client=az_model_client,
    tools=[ai_search_tool],
    system_message="Use tools to solve tasks.",
)

print("AI Search Agent created")

AI Search Agent created


### Create Bing Search Agent

In [110]:
from azure.ai.projects.models import BingGroundingTool

async def web_ai_agent(query: str) -> str:
    print("This is Bing for Azure AI Agent Service .......")
    bing = BingGroundingTool(connection_id=conn_id)
    with project_client:
        agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="bing-search-assistant",
            instructions="""        
                You are a web search agent.
                Your only tool is search_tool - use it to find information.
                You make only one search call at a time.
                Once you have the results, you never do calculations based on them.
            """,
            tools=bing.definitions,
            headers={"x-ms-enable-preview": "true"}
        )
        
        print(f"Created agent, ID: {agent.id}")

        # Create thread for communication
        thread = project_client.agents.create_thread()
        print(f"Created thread, ID: {thread.id}")

        # Create message to thread
        message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=query,
        )
        print(f"SMS: {message}")
        # Create and process agent run in thread with tools
        run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
        print(f"Run finished with status: {run.status}")

        if run.status == "failed":
            print(f"Run failed: {run.last_error}")

        # Delete the assistant when done
        project_client.agents.delete_agent(agent.id)
        print("Deleted agent")

        # Fetch and log all messages
        messages = project_client.agents.list_messages(thread_id=thread.id)
        print("Messages:"+ messages["data"][0]["content"][0]["text"]["value"])
    return messages["data"][0]["content"][0]["text"]["value"]

In [111]:
bing_search_agent = AssistantAgent(
    name="assistant",
    model_client=az_model_client,
    tools=[web_ai_agent],
    system_message="Use tools to solve tasks.",
)

print("Bing Search Agent created")

Bing Search Agent created


In [112]:
async def save_blog_agent(blog_content: str) -> str:

    print("This is Code Interpreter for Azure AI Agent Service .......")


    code_interpreter = CodeInterpreterTool()
        
    agent = project_client.agents.create_agent(
            model="gpt-4o-mini",
            name="my-agent",
            instructions="You are helpful agent",
            tools=code_interpreter.definitions,
            # tool_resources=code_interpreter.resources,
    )

    thread = project_client.agents.create_thread()

    message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content="""
        
                    You are my Python programming assistant. Generate code,save """+ blog_content +
                    
                """    
                    and execute it according to the following requirements

                    1. Save blog content to blog-{YYMMDDHHMMSS}.md

                    2. give me the download this file link
                """,
    )
    # create and execute a run
    run = project_client.agents.create_and_process_run(thread_id=thread.id, assistant_id=agent.id)
    print(f"Run finished with status: {run.status}")

    if run.status == "failed":
            # Check if you got "Rate limit is exceeded.", then you want to get more quota
        print(f"Run failed: {run.last_error}")

        # # delete the original file from the agent to free up space (note: this does not delete your version of the file)
        # project_client.agents.delete_file(file.id)
        # print("Deleted file")

        # print the messages from the agent
    messages = project_client.agents.get_messages(thread_id=thread.id)
    print(f"Messages: {messages}")

        # get the most recent message from the assistant
    last_msg = messages.get_last_text_message_by_sender("assistant")
    if last_msg:
        print(f"Last Message: {last_msg.text.value}")

        # print(f"File: {messages.file_path_annotations}")


    for file_path_annotation in messages.file_path_annotations:

        file_name = os.path.basename(file_path_annotation.text)

        project_client.agents.save_file(file_id=file_path_annotation.file_path.file_id, file_name=file_name,target_dir="./blog")
        

    project_client.agents.delete_agent(agent.id)
    print("Deleted agent")

    # project_client.close()

    return "Saved"

In [113]:
save_blog_content_agent = AssistantAgent(
    name="save_post_content_agent",
    model_client=az_model_client,
    tools=[save_blog_agent],
    system_message="""
        Save post content. Respond with 'Saved' to when your post are saved.
    """
)

In [114]:
write_agent = AssistantAgent(
    name="write_agent",
    model_client=az_model_client,
    system_message="""
        You are a linked in post writer, please help me write a linked post based on research paper and bing search content."
    """
)

In [ ]:
text_termination = TextMentionTermination("Saved")
# Define a termination condition that stops the task after 5 messages.
max_message_termination = MaxMessageTermination(10)
# Combine the termination conditions using the `|`` operator so that the
# task stops when either condition is met.
termination = text_termination | max_message_termination
reflection_team = RoundRobinGroupChat([ai_search_agent], termination_condition=termination)
# reflection_team = RoundRobinGroupChat([ai_search_agent, bing_search_agent, write_agent,save_blog_content_agent], termination_condition=termination)

In [116]:
await Console(
    reflection_team.run_stream(task="""
                    I'm writing a linked in post about a research paper on llm evaluations.
                    The research paper is stored in the Azure AI Search index.
                    Query the research paper using the AI Search agent. 
                    Generate a high-engagement Linked In post about the challenges and solutions in the research paper.
                    Identify Key Components:
                    - Problem/Research Question: What is the research trying to address or investigate? 
                    - Methods/Approach: How did the researchers conduct their study? 
                    - Results/Findings: What did the research uncover? 
                    - Conclusions/Implications: What are the main takeaways and significance of the research? 
                    - Limitations and Future Directions: What
                    Extract keywords and use bing search to explain the keywords in the research paper.                    
                    The tone should be authoritative yet engaging. 
                    Follow the Hook → Context → Insights → Engagement Trigger structure. End with a question to spark discussion. Add 3-5 relevant hashtags.

    """)
)  

---------- user ----------

                    I'm writing a linked in post about a research paper on llm evaluations.
                    The research paper is stored in the Azure AI Search index.
                    Query the research paper using the AI Search agent. 
                    Generate a high-engagement Linked In post about the challenges and solutions in the research paper.
                    Identify Key Components:
                    - Problem/Research Question: What is the research trying to address or investigate? 
                    - Methods/Approach: How did the researchers conduct their study? 
                    - Results/Findings: What did the research uncover? 
                    - Conclusions/Implications: What are the main takeaways and significance of the research? 
                    - Limitations and Future Directions: What
                    Extract keywords and use bing search to explain the keywords in the research paper.                    
   

Error processing publish message for ai_search_agent_b15dbb5d-a9e3-4b14-bfab-f1bf174ce877/b15dbb5d-a9e3-4b14-bfab-f1bf174ce877
Traceback (most recent call last):
  File "c:\code\sriksml\.venv\Lib\site-packages\httpx\_transports\default.py", line 101, in map_httpcore_exceptions
    yield
  File "c:\code\sriksml\.venv\Lib\site-packages\httpx\_transports\default.py", line 394, in handle_async_request
    resp = await self._pool.handle_async_request(req)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\code\sriksml\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 216, in handle_async_request
    raise exc from None
  File "c:\code\sriksml\.venv\Lib\site-packages\httpcore\_async\connection_pool.py", line 196, in handle_async_request
    response = await connection.handle_async_request(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\code\sriksml\.venv\Lib\site-packages\httpcore\_async\connection.py", line 99, in handle_async_request
    rais

APITimeoutError: Request timed out.